In [6]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.applications import imagenet_utils
from keras.preprocessing.image import *
from keras.utils import multi_gpu_model

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from glob import glob
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import *

import multiprocessing
from multiprocessing.dummy import Pool
from pprint import pprint

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

In [14]:
batch_size = 64
preprocess_func = inception_resnet_v2.preprocess_input
MODEL = InceptionResNetV2

width = 334
height = 334
classes = ['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']
n_classes = len(classes)

In [15]:
idg2 = ImageDataGenerator()
gen_valid = idg2.flow_from_directory('crop_valid_64/', target_size=(height, width), shuffle=False, 
                                     interpolation='lanczos', classes=classes, batch_size=batch_size)
y_valid = (gen_valid.classes != 0).astype(int)

Found 190008 images belonging to 10 classes.


In [4]:
input_tensor = Input((height, width, 3))

x = Lambda(preprocess_func)(input_tensor)
base_model = MODEL(include_top=False, weights='imagenet', input_tensor=x)

x = GlobalAvgPool2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(n_classes, activation='softmax')(x)
model = Model(input_tensor, x)

In [7]:
parallel_model = multi_gpu_model(model, gpus=4)

In [8]:
glob('models/irv2_crop_0726-0.000001-*')

[]

In [9]:
gen_valid.filenames

['正常/J01_2018.06.13 13_24_39_0_0.png',
 '正常/J01_2018.06.13 13_24_39_0_1024.png',
 '正常/J01_2018.06.13 13_24_39_0_1088.png',
 '正常/J01_2018.06.13 13_24_39_0_1152.png',
 '正常/J01_2018.06.13 13_24_39_0_1216.png',
 '正常/J01_2018.06.13 13_24_39_0_128.png',
 '正常/J01_2018.06.13 13_24_39_0_1280.png',
 '正常/J01_2018.06.13 13_24_39_0_1344.png',
 '正常/J01_2018.06.13 13_24_39_0_1408.png',
 '正常/J01_2018.06.13 13_24_39_0_1472.png',
 '正常/J01_2018.06.13 13_24_39_0_1536.png',
 '正常/J01_2018.06.13 13_24_39_0_1586.png',
 '正常/J01_2018.06.13 13_24_39_0_192.png',
 '正常/J01_2018.06.13 13_24_39_0_256.png',
 '正常/J01_2018.06.13 13_24_39_0_320.png',
 '正常/J01_2018.06.13 13_24_39_0_384.png',
 '正常/J01_2018.06.13 13_24_39_0_448.png',
 '正常/J01_2018.06.13 13_24_39_0_512.png',
 '正常/J01_2018.06.13 13_24_39_0_576.png',
 '正常/J01_2018.06.13 13_24_39_0_64.png',
 '正常/J01_2018.06.13 13_24_39_0_640.png',
 '正常/J01_2018.06.13 13_24_39_0_704.png',
 '正常/J01_2018.06.13 13_24_39_0_768.png',
 '正常/J01_2018.06.13 13_24_39_0_832.png',
 '正常/J01_

# 统计每个图片切割了多少个

In [18]:
fnames = gen_valid.filenames
len(fnames)

190008

In [27]:
def get_main(a):
    return a[:a.rfind('_', 0, a.rfind('_'))]

In [28]:
from collections import Counter
Counter(map(get_main, fnames))

Counter({'正常/J01_2018.06.13 13_24_39': 936,
         '正常/J01_2018.06.13 13_27_54': 936,
         '正常/J01_2018.06.13 14_41_29': 936,
         '正常/J01_2018.06.13 14_44_12': 936,
         '正常/J01_2018.06.13 14_47_09': 936,
         '正常/J01_2018.06.16 08_47_33': 936,
         '正常/J01_2018.06.16 08_49_15': 936,
         '正常/J01_2018.06.16 09_18_33': 936,
         '正常/J01_2018.06.16 09_25_55': 936,
         '正常/J01_2018.06.16 09_41_45': 936,
         '正常/J01_2018.06.16 10_22_48': 936,
         '正常/J01_2018.06.16 10_36_08': 936,
         '正常/J01_2018.06.17 09_51_21': 936,
         '正常/J01_2018.06.17 10_20_27': 936,
         '正常/J01_2018.06.17 13_23_04': 936,
         '正常/J01_2018.06.17 13_48_27': 936,
         '正常/J01_2018.06.17 13_53_53': 936,
         '正常/J01_2018.06.17 14_09_56': 936,
         '正常/J01_2018.06.17 14_19_57': 936,
         '正常/J01_2018.06.17 14_48_02': 936,
         '正常/J01_2018.06.17 15_34_09': 936,
         '正常/J01_2018.06.17 15_38_47': 936,
         '正常/J01_2018.06.19 09_3

# 进行预测

In [12]:
glob('models/irv2_0726-0.000001-%d*' % 35)

['models/irv2_0726-0.000001-35-0.581200.h5']

In [16]:
path = glob('models/irv2_0726-0.000001-%d*' % 35)[0]
model.load_weights(path)

y_pred = parallel_model.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())
y_preds.append(y_pred)

y_pred2 = 1 - y_pred[:,0]
y_pred3 = 1 - (np.argmax(y_pred, axis=-1) == 0)
pprint(roc_auc_score(y_valid, y_pred2))
pprint(accuracy_score(y_valid, y_pred3))
pprint(confusion_matrix(y_valid > 0.5, y_pred3))

743/743 [==============================] - 846s 1s/step


NameError: name 'y_preds' is not defined

# 计算 AUC

In [34]:
n1 = 936

y_valid = (gen_valid.classes[::n1] != 0).astype(int)
y_valid

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1])

In [92]:
y_pred2 = y_pred.reshape((-1, 936, 10))

In [96]:
y_pred2.argmax(axis=-1)[6]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [94]:
y_pred2.argmax(axis=-1)[-1]

array([0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 9, 9, 0, 9, 9,
       9, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 9, 9, 0, 0, 0, 3, 3, 3, 3, 3, 3,
       0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 3, 0, 9, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 9, 3, 3, 3, 3,
       3, 0, 0, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9, 0,
       9, 9, 9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3,
       3, 0, 0, 0, 3, 3, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,

In [67]:
y_pred3 = list(map(lambda x:Counter(x.argmax(axis=-1))[0] / 936, y_pred2))

In [88]:
roc_auc_score(y_valid, 1-np.array(y_pred3))

0.9079172001707214

In [69]:
roc_auc_score(y_valid, 1-y_pred2.mean(axis=1)[:,0])

0.889991463935126

In [90]:
roc_auc_score(y_valid, 1-y_pred2.max(axis=1)[:,0])

0.5603393085787451